<a href="https://colab.research.google.com/github/usheikh2/daen690/blob/main/combine_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 48 kB 5.0 MB/s 
     |████████████████████████████████| 4.1 MB 47.7 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=9c351ec1042ccb01689b8e683dc1575cb4a400a51d5682804d5feb232935fd9f
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


In [ ]:
import os
import pathlib
import random
import numpy as np
import tensorflow as tf
import cv2
# Import the object detection module.
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util

In [ ]:
tf.gfile = tf.io.gfile

model_path_bb = "/content/drive/MyDrive/bobs_builders_data/run4_output/final_model_4"
model_path_hh = "/content/drive/MyDrive/HH_Final_Model/Final_Model"
labels_path_bb = "/content/drive/MyDrive/bobs_builders_data/valid/EquipmentType_label_map.pbtxt"
labels_path_hh = "/content/drive/MyDrive/hh_label_map.pbtxt"
image_dir = "/content/drive/MyDrive/Combined_Images"
image_file_pattern = "*.jpg"
output_path="/content/drive/MyDrive/Combine_ObjDetection"

In [ ]:
PATH_TO_IMAGES_DIR = pathlib.Path(image_dir)
IMAGE_PATHS = sorted(list(PATH_TO_IMAGES_DIR.glob(image_file_pattern)))

In [ ]:
category_index_bb = label_map_util.create_category_index_from_labelmap(labels_path_bb, use_display_name=True)
category_index_hh = label_map_util.create_category_index_from_labelmap(labels_path_hh, use_display_name=True)
class_num_bb =len(category_index_bb)
class_num_hh =len(category_index_hh)

def get_color_table(class_num, seed=0):
    random.seed(seed)
    color_table = {}
    for i in range(class_num):
        color_table[i] = [random.randint(0, 255) for _ in range(3)]
    return color_table

colortable_bb = get_color_table(class_num_bb)
colortable_hh = get_color_table(class_num_hh)

# # Model preparation and loading the model from the disk
def load_model(model_path):

    model_dir = pathlib.Path(model_path) / "saved_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']
    # print(model)
    return model


# Predict objects and bounding boxes and format the result
def run_inference_for_single_image(model, image):

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run prediction from the model
    output_dict = model(input_tensor) 

    # Input to model is a tensor, so the output is also a tensor
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    # print("number of detection: ", num_detections)
    output_dict = {key: value[0, :num_detections].numpy()
                   for key, value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    # print("output dict: ", output_dict)
    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    return output_dict


tot_scores = []

def visualize(output_dict, category_index, colortable, image_np):
    # Visualization of the results of a detection.
    for i in range(output_dict['detection_classes'].size):

        box = output_dict['detection_boxes'][i]
        classes = output_dict['detection_classes'][i]
        scores = output_dict['detection_scores'][i]
        print("box: ", box, "classes: ",classes, "scores: ", scores)
        tot_scores.append(scores)

        if scores > 0.5:
            h = image_np.shape[0]
            w = image_np.shape[1]
            classname = category_index[classes]['name']
            classid = category_index[classes]['id']
            print("h: ", h, "w: ", w, "classname: ", classname, "classid: ", classid)
            #Draw bounding boxes
            cv2.rectangle(image_np, (int(box[1] * w), int(box[0] * h)), (int(box[3] * w), int(box[2] * h)), colortable[classid - 1], 2)

            #Write the class name on top of the bounding box
            font = cv2.FONT_HERSHEY_COMPLEX_SMALL
            size = cv2.getTextSize(str(classname) + ":" + str(scores), font, 0.75, 1)[0][0]

            cv2.rectangle(image_np,(int(box[1] * w), int(box[0] * h-20)), ((int(box[1] * w)+size+5), int(box[0] * h)), colortable[classid - 1],-1)
            cv2.putText(image_np, str(classname) + ":" + str(scores),
                    (int(box[1] * w), int(box[0] * h)-5), font, 0.75, (0,0,0), 1, 1)
        else:
            break

def infer_object(model1, model2, image_path):
    # Read the image using openCV and create an image numpy
    # The final output image with boxes and labels on it.
    imagename = os.path.basename(image_path)

    image_np = cv2.imread(os.path.abspath(image_path))
    # Actual detection.
    output_dict_bb = run_inference_for_single_image(model1, image_np)
    output_dict_hh = run_inference_for_single_image(model2, image_np)

    # Visualize 
    visualize(output_dict_bb, category_index_bb, colortable_bb, image_np)
    visualize(output_dict_hh, category_index_hh, colortable_hh, image_np)
    
    # Save the result image with bounding boxes and class labels in file system
    cv2.imwrite(output_path+"/"+imagename, image_np)
    # cv2.imshow(imagename, image_np)

In [ ]:
# Obtain the model object
detection_model1 = load_model(model_path_bb)
detection_model2 = load_model(model_path_hh)

In [ ]:
# For each image, call the prediction

for image_path in IMAGE_PATHS:
      infer_object(detection_model1, detection_model2, image_path)

box:  [0.27001753 0.49884808 0.7870203  0.8139696 ] classes:  6 scores:  0.7187605
h:  630 w:  1200 classname:  excavator classid:  6
box:  [0.3180643  0.14129794 0.8202982  0.7267167 ] classes:  8 scores:  0.6194376
h:  630 w:  1200 classname:  mobile_crane classid:  8
box:  [0.27404296 0.35367543 0.8026513  0.80164665] classes:  8 scores:  0.48108545
box:  [0.39008948 0.20908822 0.41358885 0.2195542 ] classes:  1 scores:  0.25614452
box:  [0.12294917 0.40188602 0.40703642 0.6590769 ] classes:  5 scores:  0.24847862
box:  [0.09641809 0.12023706 0.15667783 0.16506046] classes:  2 scores:  0.19003296
box:  [0.03638119 0.26904494 0.8586635  0.62129825] classes:  9 scores:  0.2832486
box:  [0.60341483 0.9920786  0.62499267 0.9993815 ] classes:  1 scores:  0.18548444
box:  [0.34569088 0.14341566 1.         0.78416693] classes:  1 scores:  0.20715553
box:  [0.42995536 0.5048444  0.47523713 0.5392859 ] classes:  2 scores:  0.55041856
h:  662 w:  1000 classname:  person classid:  2
box:  [0.5

In [ ]:
# print(tot_scores)
print(sum(tot_scores)/len(tot_scores))


abovehalf = []
for i in range(len(tot_scores)):
  if tot_scores[i] > 0.5:
    abovehalf.append(tot_scores[i])

print(abovehalf)
print(sum(abovehalf)/len(abovehalf))

0.4031204437570913
[0.7187605, 0.6194376, 0.53507054, 0.851874, 0.68541014, 0.92297506, 0.6189786, 0.71485925, 0.5533261, 0.8055146, 0.5264637, 0.7398809, 0.58505666, 0.8921373]
0.6978389195033482
